# Data

In [33]:
# ────────────────────────────────────────────────────────────────────────────────
# 0. Imports & reproducibility
# ────────────────────────────────────────────────────────────────────────────────
import math, random, numpy as np, torch
from torch import nn
# Basic
import torch
import math
import torch.nn as nn
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau
import numpy as np
from numpy import random
torch.set_num_threads(4)

# PDEi
from utils_pde.utils_pde_2dpoisson import Poisson2D

# Viz
from utils_tools.utils_result_viz import plot_predictions_2D
from utils_tools.utils_tuning import save_plot

# Base Mdoels
from utils_uqmd.utils_uq_dropout import DropoutPINN
from utils_uqmd.utils_uq_mlp import MLPPINN
from utils_uqmd.utils_uq_vi import VIBPINN
from utils_uqmd.utils_uq_distance import DistanceUQPINN

# CP
from utils_uqmd.utils_uq_cp import CP

torch.set_num_threads(4)
seed = 59
random.seed(seed); np.random.seed(seed); torch.manual_seed(seed)
if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)

# ────────────────────────────────────────────────────────────────────────────────
# 1. Ground-truth PDE setup 
# ────────────────────────────────────────────────────────────────────────────────
from utils_pde.utils_pde_allencahn2d import AllenCahn2D

# 1) keep the torch-based definition
def true_solution(xy):
    return torch.sin(math.pi*xy[...,0:1]) * torch.sin(math.pi*xy[...,1:2])

domain=((-1.0, 1.0), (-1.0, 1.0))
pde = AllenCahn2D(lam=0.05, domain=domain, true_solution=true_solution)

# ────────────────────────────────────────────────────────────────────────────────
# 2. Data Generation (Noisy data for training, test, calibration)
# ────────────────────────────────────────────────────────────────────────────────
colloc_pt_num = 250  # Number of collocation points

data_noise = 0.05
N_train = 500
N_test = 200
N_calib = 100
N_valid = 100

N_total= N_train+N_test+N_calib

X_train, Y_train = pde.data_generation(N_train, data_noise)
X_test, Y_test = pde.data_generation(N_test, data_noise)
X_calibration, Y_calibration = pde.data_generation(N_calib, data_noise)
X_validation, Y_validation= pde.data_generation(N_valid, data_noise)

X_vis, Y_vis = pde.data_generation(N_total, data_noise)
# Generating alphas to test
from utils_tools.utils_result_metrics import generating_alphas
alphas = generating_alphas(20)

alpha = 0.95

# ────────────────────────────────────────────────────────────────────────────────
# 3. Visualization: Plot ground-truth & noisy data
# ────────────────────────────────────────────────────────────────────────────────

def make_dense_grid(domain, N=200):
    x = np.linspace(domain[0][0], domain[0][1], N)
    y = np.linspace(domain[1][0], domain[1][1], N)
    X, Y = np.meshgrid(x, y)
    XY_grid = np.stack([X.ravel(), Y.ravel()], axis=-1)
    return XY_grid, X, Y

import matplotlib.pyplot as plt

def plot_truth_and_samples_2D(
    X_train, Y_train, grid, U_true_grid, domain,
    title="Allen-Cahn 2D: ground-truth vs noisy data"
):
    fig, ax = plt.subplots(figsize=(7, 6))
    # Show the true solution as a colormap
    x = np.linspace(domain[0][0], domain[0][1], U_true_grid.shape[0])
    y = np.linspace(domain[1][0], domain[1][1], U_true_grid.shape[1])
    im = ax.imshow(
        U_true_grid,
        extent=(domain[0][0], domain[0][1], domain[1][0], domain[1][1]),
        origin='lower',
        aspect='auto',
        alpha=0.8,
        cmap='coolwarm'
    )
    # Overlay noisy training points
    ax.scatter(X_train[:,0], X_train[:,1], c=Y_train, edgecolor='k', cmap='viridis', s=18, label="Noisy samples")
    plt.colorbar(im, ax=ax, label="u(x, y)")
    ax.set_xlabel("x"); ax.set_ylabel("y")
    ax.set_title(title)
    ax.legend()
    plt.tight_layout()



# 4) plot again
grid_test, X_mesh, Y_mesh = make_dense_grid(domain, 200)
grid_test_torch = torch.tensor(grid_test, dtype=torch.float32)
U_true_grid = true_solution(grid_test_torch).numpy().reshape(X_mesh.shape)

save_plot(
    plot_truth_and_samples_2D,
    save_dir="2D_AllenCahn", prefix="2D_AllenCahn_data",
)(
    X_vis, Y_vis, grid_test, U_true_grid, domain,
    title="Allen-Cahn 2D: ground-truth vs noisy data"
)


# --------------------------------------------
# Defining Testing
# --------------------------------------------
n_grid = 500
x = torch.linspace(1.0, -1.0, n_grid)
y = torch.linspace(1.0, -1.0, n_grid)
X, Y = torch.meshgrid(x, y, indexing='xy')
grid_test = torch.cat([X.reshape(-1, 1), Y.reshape(-1, 1)], dim=1)


<Figure size 640x480 with 0 Axes>

# Feature Distance Model

## Define Model

In [2]:
from utils_uqmd.utils_uq_distance import DistanceUQPINN

# Base-Model Instance
model_args = {
    "pde_class":pde,
    "input_dim":2,
    "hidden_dims":[16, 32, 64, 64, 64, 32, 16], 
    "output_dim":1,
}

# CP-Model

cp_testing_args = {
    "alphas":alphas, 
    "X_test":X_test, "Y_test":Y_test, 
    "X_cal":X_calibration, "Y_cal":Y_calibration, 
    "X_train":X_train, "Y_train":Y_train, 
    "heuristic_u":"feature", # Change base on if the baseline cp
    "k":10
}

baseline_testing_args = { 
    # "uqmodel":dist_pinn
    "alphas":alphas, 
    "X_test":X_test, "Y_test":Y_test,
    "heuristic_u":"feature",
    "n_samples":10, 
}

dist_feat_pinn = DistanceUQPINN(**model_args)

## Training Base Model

In [3]:
# Base Model
# Training
fit_args = dict(
    coloc_pt_num=colloc_pt_num, 
    X_train=X_train, 
    Y_train=Y_train
)
fit_kwargs_grid = dict(
    epochs=5000,
    λ_pde=1.0, λ_ic=5.0, λ_data=1.0,
    lr=1e-3, stop_schedule=20000
)
baseline_loss_dict = dist_feat_pinn.fit(**fit_args, **fit_kwargs_grid)

# Test the model performance
baseline_data_loss = dist_feat_pinn.data_loss(X_validation, Y_validation)


# Predicting
baseline_pred_kwargs = dict(
    n_samples=10, 
    heuristic_u="feature"
)
dist_feat_pinn_uncal_predset = dist_feat_pinn.predict(alpha, grid_test, 
                                   **baseline_pred_kwargs)


ep     1 | L=1.34e+00 | lr=1.0e-03
ep   500 | L=1.79e-01 | lr=1.0e-03
ep  1000 | L=1.03e-01 | lr=1.0e-03
ep  1500 | L=8.65e-02 | lr=1.0e-03
ep  2000 | L=1.12e-01 | lr=1.0e-03
ep  2500 | L=3.62e-02 | lr=1.0e-03
ep  3000 | L=3.71e-02 | lr=1.0e-03
ep  3500 | L=3.49e-02 | lr=1.0e-03
ep  4000 | L=2.04e-02 | lr=1.0e-03
ep  4500 | L=3.61e-02 | lr=1.0e-03
ep  5000 | L=1.20e-02 | lr=1.0e-03


In [4]:
dist_feat_pinn_uncal_predset = dist_feat_pinn.predict(alpha, grid_test, 
                                   **baseline_pred_kwargs)

## Create CP Model

In [5]:
# CP Model
dist_feat_pinn_cp = CP(dist_feat_pinn)

# Predicting
cp_pred_kwargs = {
        "X_train":X_train,  "Y_train":Y_train,
        "X_cal":X_calibration, "Y_cal":Y_calibration,
        "heuristic_u":"feature",  # Change this based on cp
        "k":10
}

dist_feat_pinn_cp_cal_predset = dist_feat_pinn_cp.predict(
        alpha=alpha, X_test=grid_test,
        **cp_pred_kwargs
)

## Plot Coverage Plot

In [6]:
# Computing Coverage Info
from utils_tools.utils_result_metrics import cp_test_uncertainties, vi_test_uncertainties, do_test_uncertainties, dist_test_uncertainties
dist_feat_pinn_df_uncal = dist_test_uncertainties(uqmodel=dist_feat_pinn, **baseline_testing_args)
dist_feat_pinndf_cal = cp_test_uncertainties(dist_feat_pinn_cp, **cp_testing_args)

# Save the plot
from utils_tools.utils_tuning import save_plot
from utils_tools.utils_result_viz import plot_2D_comparison_with_coverage, plot_metrics_table
save_plot(
    plot_2D_comparison_with_coverage,
    save_dir="2D_AllenCahn", prefix="feature_distance",
    loss=baseline_data_loss
)(
    grid_test, dist_feat_pinn_uncal_predset, dist_feat_pinn_cp_cal_predset, 
    true_solution, 
    dist_feat_pinn_df_uncal, dist_feat_pinndf_cal,
    title="2D UQ", main_title="Feature Distance Model", 
    X_vis=X_train, Y_vis=Y_train
)



100%|██████████| 19/19 [00:01<00:00, 16.63it/s]


<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

In [24]:
save_plot(
    plot_metrics_table,
    save_dir="2D_AllenCahn", prefix="feature_distance_table",
    loss=baseline_data_loss
)(
    grid_test, dist_feat_pinn_uncal_predset, dist_feat_pinn_cp_cal_predset, 
    true_solution, 
    dist_feat_pinn_df_uncal, dist_feat_pinndf_cal,
    title="2D UQ", main_title="Feature Distance Model Metrics", 
    X_vis=X_train, Y_vis=Y_train
)

<Figure size 640x480 with 0 Axes>

# Latent Distance Model

## Define Model

In [7]:
from utils_uqmd.utils_uq_distance import DistanceUQPINN

# Base-Model Instance
model_args = {
    "pde_class":pde,
    "input_dim":2,
    "hidden_dims":[16, 32, 64, 64, 64, 32, 16], 
    "output_dim":1,
}

baseline_pred_kwargs = dict(n_samples=200)


# CP-Model

cp_testing_args = {
    "alphas":alphas, 
    "X_test":X_test, "Y_test":Y_test, 
    "X_cal":X_calibration, "Y_cal":Y_calibration, 
    "X_train":X_train, "Y_train":Y_train, 
    "heuristic_u":"latent", # Change base on if the baseline cp
    "k":10
}

baseline_testing_args = { 
    # "uqmodel":dist_pinn
    "alphas":alphas, 
    "X_test":X_test, "Y_test":Y_test,
    "heuristic_u":"latent",
    "n_samples":10, 
}

dist_lat_pinn = DistanceUQPINN(**model_args)

## Training Base Model

In [8]:
# Base Model
# Training
fit_args = dict(
    coloc_pt_num=colloc_pt_num, 
    X_train=X_train, 
    Y_train=Y_train
)
fit_kwargs_grid = dict(
    epochs=5000,
    λ_pde=1.0, λ_ic=5.0, λ_data=1.0,
    lr=1e-3, stop_schedule=20000
)
baseline_loss_dict = dist_lat_pinn.fit(**fit_args, **fit_kwargs_grid)

# Test the model performance
baseline_data_loss = dist_lat_pinn.data_loss(X_validation, Y_validation)


# Predicting
baseline_pred_kwargs = dict(
    n_samples=10, 
    heuristic_u="latent"
)
dist_lat_pinn_uncal_predset = dist_lat_pinn.predict(alpha, grid_test, 
                                   **baseline_pred_kwargs)


ep     1 | L=1.18e+00 | lr=1.0e-03
ep   500 | L=1.89e-01 | lr=1.0e-03
ep  1000 | L=6.10e-02 | lr=1.0e-03
ep  1500 | L=2.91e-02 | lr=1.0e-03
ep  2000 | L=2.67e-02 | lr=1.0e-03
ep  2500 | L=2.38e-02 | lr=1.0e-03
ep  3000 | L=2.88e-02 | lr=1.0e-03
ep  3500 | L=4.20e-02 | lr=1.0e-03
ep  4000 | L=1.49e-02 | lr=1.0e-03
ep  4500 | L=1.55e-02 | lr=1.0e-03
ep  5000 | L=1.39e-02 | lr=1.0e-03


## Create CP Model

In [9]:
# CP Model
dist_lat_pinn_cp = CP(dist_lat_pinn)

# Predicting
cp_pred_kwargs = {
        "X_train":X_train,  "Y_train":Y_train,
        "X_cal":X_calibration, "Y_cal":Y_calibration,
        "heuristic_u":"latent",  # Change this based on cp
        "k":10
}

dist_lat_pinn_cp_cal_predset = dist_lat_pinn_cp.predict(
        alpha=alpha, X_test=grid_test,
        **cp_pred_kwargs
)

## Plot Coverage Plot

In [10]:
# Computing Coverage Info
from utils_tools.utils_result_metrics import cp_test_uncertainties, vi_test_uncertainties, do_test_uncertainties, dist_test_uncertainties
dist_lat_pinn_df_uncal = dist_test_uncertainties(uqmodel=dist_lat_pinn, **baseline_testing_args)
dist_lat_pinn_df_cal = cp_test_uncertainties(dist_lat_pinn_cp, **cp_testing_args)

# Save the plot
from utils_tools.utils_tuning import save_plot
from utils_tools.utils_result_viz import plot_2D_comparison_with_coverage
save_plot(
    plot_2D_comparison_with_coverage,
    save_dir="2D_AllenCahn", prefix="latent_distance",
    loss=baseline_data_loss
)(
    grid_test, dist_lat_pinn_uncal_predset, dist_lat_pinn_cp_cal_predset, 
    true_solution, 
    dist_lat_pinn_df_uncal, dist_lat_pinn_df_cal,
    title="2D UQ", main_title="Latent Distance Model", 
    X_vis=X_train, Y_vis=Y_train
)


100%|██████████| 19/19 [00:01<00:00, 15.48it/s]


<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

In [25]:

save_plot(
    plot_metrics_table,
    save_dir="2D_AllenCahn", prefix="latent_distance_table",
    loss=baseline_data_loss
)(
    grid_test, dist_lat_pinn_uncal_predset, dist_lat_pinn_cp_cal_predset, 
    true_solution, 
    dist_lat_pinn_df_uncal, dist_lat_pinn_df_cal,
    title="2D UQ", main_title="Latent Distance Model Metrics", 
    X_vis=X_train, Y_vis=Y_train
)

<Figure size 640x480 with 0 Axes>

# Dropout Model

## Define Model

In [11]:
from utils_uqmd.utils_uq_dropout import DropoutPINN

# Base Model Instance
model_args = {
    "pde_class":pde,
    "input_dim":2,
    "hidden_dims":[16, 32, 64, 64, 64, 32, 16], 
    "output_dim":1,
}

# CP Model
cp_testing_args = {
    "alphas":alphas, 
    "X_test":X_test, "Y_test":Y_test, 
    "X_cal":X_calibration, "Y_cal":Y_calibration, 
    "X_train":X_train, "Y_train":Y_train, 
    "heuristic_u":"raw_std", # Change base on if the baseline cp
    "k":10
}

cp_pred_kwargs = {
    "X_train":X_train,  "Y_train":Y_train,
    "X_cal":X_calibration, "Y_cal":Y_calibration,
    "heuristic_u":"raw_std",
    "k":10
}

baseline_testing_args = { 
    # "uqmodel":do_pinn,   # Change this
    "alphas":alphas, 
    "X_test":X_test, "Y_test":Y_test,
    "n_samples":20
}

dropout_pinn = DropoutPINN(**model_args)

## Training Base Model

In [12]:
# Base Model
# Training
fit_args = dict(
    coloc_pt_num=colloc_pt_num, 
    X_train=X_train, 
    Y_train=Y_train
)
fit_kwargs_grid = dict(
    epochs=5000,
    λ_pde=1.0, λ_ic=5.0, λ_data=1.0,
    lr=1e-3, stop_schedule=20000
)
baseline_loss_dict = dropout_pinn.fit(**fit_args, **fit_kwargs_grid)

# Test the model performance
baseline_data_loss = dropout_pinn.data_loss(X_validation, Y_validation)


# Predicting
baseline_pred_kwargs = dict(
    n_samples=20
)
dropout_pinn_uncal_predset = dropout_pinn.predict(alpha, grid_test, 
                                   **baseline_pred_kwargs)


ep     1 | L=1.12e+00 | data=2.75e-01 | pde=5.31e-01  ic=0.00e+00  bc=3.10e-02 | lr=1.00e-03
ep   500 | L=4.14e-01 | data=1.06e-01 | pde=2.05e-01  ic=0.00e+00  bc=1.03e-02 | lr=1.00e-03
ep  1000 | L=3.05e-01 | data=7.61e-02 | pde=1.41e-01  ic=0.00e+00  bc=8.82e-03 | lr=1.00e-03
ep  1500 | L=2.60e-01 | data=6.50e-02 | pde=1.16e-01  ic=0.00e+00  bc=7.98e-03 | lr=1.00e-03
ep  2000 | L=2.74e-01 | data=6.79e-02 | pde=1.32e-01  ic=0.00e+00  bc=7.42e-03 | lr=1.00e-03
ep  2500 | L=2.37e-01 | data=5.76e-02 | pde=1.13e-01  ic=0.00e+00  bc=6.68e-03 | lr=1.00e-03
ep  3000 | L=2.19e-01 | data=5.43e-02 | pde=1.06e-01  ic=0.00e+00  bc=5.84e-03 | lr=1.00e-03
ep  3500 | L=2.08e-01 | data=5.66e-02 | pde=9.73e-02  ic=0.00e+00  bc=5.39e-03 | lr=1.00e-03
ep  4000 | L=2.00e-01 | data=4.58e-02 | pde=9.03e-02  ic=0.00e+00  bc=6.37e-03 | lr=1.00e-03
ep  4500 | L=2.06e-01 | data=4.90e-02 | pde=1.06e-01  ic=0.00e+00  bc=5.13e-03 | lr=1.00e-03
ep  5000 | L=2.07e-01 | data=5.23e-02 | pde=1.12e-01  ic=0.00e+00  bc=

/Users/yifanyu/miniconda3/envs/cp/lib/python3.10/site-packages/torch/utils/_device.py:104: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return func(*args, **kwargs)


## Create CP Model

In [13]:
# CP Model
dropout_pinn_cp = CP(dropout_pinn)

# Predicting
cp_pred_kwargs = {
        "X_train":X_train,  "Y_train":Y_train,
        "X_cal":X_calibration, "Y_cal":Y_calibration,
        "heuristic_u":"raw_std",  # Change this based on cp
        "k":10
}

dropout_pinn_cp_cal_predset = dropout_pinn_cp.predict(
        alpha=alpha, X_test=grid_test,
        **cp_pred_kwargs
)

## Plot Coverage Plot

In [14]:
# Computing Coverage Info
from utils_tools.utils_result_metrics import cp_test_uncertainties, vi_test_uncertainties, do_test_uncertainties, dist_test_uncertainties
dropout_pinn_df_uncal = do_test_uncertainties(uqmodel=dropout_pinn, **baseline_testing_args)
dropout_pinn_df_cal = cp_test_uncertainties(dropout_pinn_cp, **cp_testing_args)

# Save the plot
from utils_tools.utils_tuning import save_plot
from utils_tools.utils_result_viz import plot_2D_comparison_with_coverage
save_plot(
    plot_2D_comparison_with_coverage,
    save_dir="2D_AllenCahn", prefix="dropout",
    loss=baseline_data_loss
)(
    grid_test, dropout_pinn_uncal_predset, dropout_pinn_cp_cal_predset, 
    true_solution, 
    dropout_pinn_df_uncal, dropout_pinn_df_cal,
    title="2D UQ", main_title="Dropout Model", 
    X_vis=X_train, Y_vis=Y_train
)




100%|██████████| 19/19 [00:03<00:00,  6.30it/s]


<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

In [26]:
save_plot(
    plot_metrics_table,
    save_dir="2D_AllenCahn", prefix="dropout_table",
    loss=baseline_data_loss
)(
    grid_test, dropout_pinn_uncal_predset, dropout_pinn_cp_cal_predset, 
    true_solution, 
    dropout_pinn_df_uncal, dropout_pinn_df_cal,
    title="2D UQ", main_title="Dropout Model Metrics", 
    X_vis=X_train, Y_vis=Y_train
)

<Figure size 640x480 with 0 Axes>

# VI Model

## Define Model

In [15]:
from utils_uqmd.utils_uq_vi import VIBPINN

# Base Model Instance
model_args = {
    "pde_class":pde,
    "input_dim":2,
    "hidden_dims":[16, 32, 64, 64, 64, 32, 16], 
    "output_dim":1,
    "mu_std" : 0.01, "rho" : -4, "prior_std" : 1.0, 
    "init_data_noise" : 1.0, "learn_data_noise" : False, 
    "act_func" : nn.Tanh()
}

baseline_testing_args = { 
    # "uqmodel":vi_model, 
    "alphas":alphas, 
    "X_test":X_test, "Y_test":Y_test
}

vi_bpinn = VIBPINN(**model_args)

## Training Base Model

In [16]:
# Base Model
# Training
fit_args = {
    "coloc_pt_num":colloc_pt_num,
    "X_train":X_train, 
    "Y_train":Y_train
}
fit_kwargs_grid = {
    "epochs":10000,
    "λ_pde":5.0, "λ_bc":5.0, "λ_elbo":1.0,  
    "lr":3e-3,
    "stop_schedule":20000
}
baseline_loss_dict = vi_bpinn.fit(**fit_args, **fit_kwargs_grid)

# Test the model performance
baseline_data_loss = vi_bpinn.data_loss(X_validation, Y_validation)


# Predicting
baseline_pred_kwargs = dict(
    n_samples=5000
)
vi_bpinn_uncal_predset = vi_bpinn.predict(alpha, grid_test, 
                                   **baseline_pred_kwargs)


ep     0 | L=1.27e+02 | elbo=1.25e+02 | pde=3.12e-01  ic=0.00e+00  bc=1.03e-04 | lr=3.00e-03 | learned noise_std=1.000e+00
ep     1 | L=1.27e+02 | elbo=1.25e+02 | pde=2.98e-01  ic=0.00e+00  bc=1.76e-04 | lr=3.00e-03 | learned noise_std=1.000e+00
ep   100 | L=1.21e+02 | elbo=1.20e+02 | pde=2.58e-01  ic=0.00e+00  bc=1.08e-04 | lr=3.00e-03 | learned noise_std=1.000e+00
ep   200 | L=1.16e+02 | elbo=1.15e+02 | pde=3.09e-01  ic=0.00e+00  bc=1.41e-04 | lr=3.00e-03 | learned noise_std=1.000e+00
ep   300 | L=1.00e+02 | elbo=9.83e+01 | pde=3.04e-01  ic=0.00e+00  bc=4.05e-02 | lr=3.00e-03 | learned noise_std=1.000e+00
ep   400 | L=7.66e+01 | elbo=7.56e+01 | pde=1.03e-01  ic=0.00e+00  bc=9.24e-02 | lr=3.00e-03 | learned noise_std=1.000e+00
ep   500 | L=7.55e+01 | elbo=7.44e+01 | pde=9.94e-02  ic=0.00e+00  bc=1.15e-01 | lr=3.00e-03 | learned noise_std=1.000e+00
ep   600 | L=7.61e+01 | elbo=7.49e+01 | pde=8.51e-02  ic=0.00e+00  bc=1.49e-01 | lr=3.00e-03 | learned noise_std=1.000e+00
ep   700 | L=8.2

## Create CP Model

In [17]:
# CP Model
vi_bpinn_cp = CP(vi_bpinn)

# CP
cp_pred_kwargs = {
        "X_train":X_train,  "Y_train":Y_train,
        "X_cal":X_calibration, "Y_cal":Y_calibration,
        "heuristic_u":"raw_std",
        "k":10
}

cp_testing_args = {
        "alphas":alphas, 
        "X_test":X_test, "Y_test":Y_test, 
        "X_cal":X_calibration, "Y_cal":Y_calibration, "X_train":X_train, "Y_train":Y_train, 
        "heuristic_u":"raw_std", # Change base on if the baseline model has its original uq band
        "k":10
}

vi_bpinn_cp_cal_predset = vi_bpinn_cp.predict(
        alpha=alpha, X_test=grid_test,
        **cp_pred_kwargs
)

## Plot Coverage Plot

In [18]:
# Computing Coverage Info
from utils_tools.utils_result_metrics import cp_test_uncertainties, vi_test_uncertainties, do_test_uncertainties, dist_test_uncertainties
vi_bpinn_df_uncal = vi_test_uncertainties(uqmodel=vi_bpinn, **baseline_testing_args)
vi_bpinn_df_cal = cp_test_uncertainties(vi_bpinn_cp, **cp_testing_args)

# Save the plot
from utils_tools.utils_tuning import save_plot
from utils_tools.utils_result_viz import plot_2D_comparison_with_coverage
save_plot(
    plot_2D_comparison_with_coverage,
    save_dir="2D_AllenCahn", prefix="vi_bpinn",
    loss=baseline_data_loss
)(
    grid_test, vi_bpinn_uncal_predset, vi_bpinn_cp_cal_predset, 
    true_solution, 
    vi_bpinn_df_uncal, vi_bpinn_df_cal,
    title="2D UQ", main_title="VI Model", 
    X_vis=X_train, Y_vis=Y_train
)



100%|██████████| 19/19 [07:13<00:00, 22.84s/it]


<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

In [27]:
save_plot(
    plot_metrics_table,
    save_dir="2D_AllenCahn", prefix="vi_bpinn_table",
    loss=baseline_data_loss
)(
    grid_test, vi_bpinn_uncal_predset, vi_bpinn_cp_cal_predset, 
    true_solution, 
    vi_bpinn_df_uncal, vi_bpinn_df_cal,
    title="2D UQ", main_title="VI Model Metrics",
    X_vis=X_train, Y_vis=Y_train
)

<Figure size 640x480 with 0 Axes>

# HMC Model

## Define Model

In [19]:
from utils_uqmd.utils_uq_hmc import HMCBPINN

# Base Model Instance
model_args = dict(
    pde_class=pde, input_dim=2, hidden_dims=[16, 32, 64, 64, 64, 32, 16],
    output_dim=1, act_func=nn.Tanh, prior_std=1.0,
    step_size=1e-3, leapfrog_steps=5
)

baseline_testing_args = {
    # "uqmodel": hmc_model,
    "alphas": alphas,
    "X_test": X_test,
    "Y_test": Y_test,
    "n_samples": 5000
}


hmc_bpinn = HMCBPINN(**model_args)

## Training Base Model

In [20]:
# Base Model
# Training
fit_args = {
    "coloc_pt_num":colloc_pt_num,
    "X_train":X_train, 
    "Y_train":Y_train
}
fit_kwargs_grid = {
    "λ_pde": 3.0,
    "λ_bc": 5.0,
    "λ_data": 1.0,
    "epochs": 5000,
    "lr":1e-3,
    "hmc_samples": 10000,
    "brun_in":5000,
    "step_size": 7e-4,
    "leapfrog_steps": 13
}
baseline_loss_dict = hmc_bpinn.fit(**fit_args, **fit_kwargs_grid)

# Test the model performance
baseline_data_loss = hmc_bpinn.data_loss(X_validation, Y_validation)

# Predict 
baseline_pred_kwargs = {"n_samples": 5000}
hmc_bpinn_uncal_predset = hmc_bpinn.predict(
            alpha=alpha, X_test=grid_test,
            **baseline_pred_kwargs
        )

MAP:  10%|█         | 521/5000 [00:03<00:31, 142.34it/s, loss=9.56e-01]

[MAP] epoch    500 −logPost=1.047e+00  Data=2.46e-01  PDE=5.51e-01  IC=0.00e+00  BC=2.29e-07


MAP:  21%|██        | 1027/5000 [00:07<00:26, 150.19it/s, loss=7.47e-01]

[MAP] epoch   1000 −logPost=7.905e-01  Data=2.46e-01  PDE=5.35e-01  IC=0.00e+00  BC=3.92e-06


MAP:  30%|███       | 1516/5000 [00:10<00:24, 139.71it/s, loss=7.44e-01]

...

HMC:  90%|█████████ | 9001/10000 [14:22<01:32, 10.85it/s, acc=0.58]

[HMC] iter   9000  acc-rate=0.58


HMC:  95%|█████████▌| 9502/10000 [15:10<00:44, 11.10it/s, acc=0.58]

[HMC] iter   9500  acc-rate=0.58


[HMC] iter  10000  acc-rate=0.57
Finished HMC: avg acceptance 0.574
Kept 9500 posterior samples


## Create CP Model

In [21]:
# CP Model
hmc_bpinn_cp = CP(hmc_bpinn)

# CP
cp_pred_kwargs = { 
    "X_train": X_train, "Y_train": Y_train,
    "X_cal": X_calibration, "Y_cal": Y_calibration,
    "heuristic_u": "raw_std", "k": 10
}

cp_testing_args = {
    "alphas":alphas, 
    "X_test":X_test, "Y_test":Y_test, 
    "X_cal":X_calibration, "Y_cal":Y_calibration, "X_train":X_train, "Y_train":Y_train, 
    "heuristic_u":"raw_std", # Change base on if the baseline model has its original uq band
    "k":10
}

hmc_bpinn_cp_cal_predset = hmc_bpinn_cp.predict(
        alpha=alpha, X_test=grid_test,
        **cp_pred_kwargs
)

## Plot Coverage Plot

In [22]:
# Computing Coverage Info
from utils_tools.utils_result_metrics import hmc_test_uncertainties, cp_test_uncertainties, vi_test_uncertainties, do_test_uncertainties, dist_test_uncertainties
hmc_bpinn_df_uncal = hmc_test_uncertainties(uqmodel=hmc_bpinn, **baseline_testing_args)
hmc_bpinn_df_cal = cp_test_uncertainties(hmc_bpinn_cp, **cp_testing_args)

# Save the plot
from utils_tools.utils_tuning import save_plot
from utils_tools.utils_result_viz import plot_2D_comparison_with_coverage
save_plot(
    plot_2D_comparison_with_coverage,
    save_dir="2D_AllenCahn", prefix="hmc_bpinn",
    loss=baseline_data_loss
)(
    grid_test, hmc_bpinn_uncal_predset, hmc_bpinn_cp_cal_predset, 
    true_solution, 
    hmc_bpinn_df_uncal, hmc_bpinn_df_cal,
    title="2D UQ", main_title="HMC Model", 
    X_vis=X_train, Y_vis=Y_train
)


  0%|          | 0/19 [00:00<?, ?it/s]/Users/yifanyu/miniconda3/envs/cp/lib/python3.10/site-packages/torch/utils/_device.py:104: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return func(*args, **kwargs)
100%|██████████| 19/19 [01:31<00:00,  4.83s/it]


<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

In [28]:

save_plot(
    plot_metrics_table,
    save_dir="2D_AllenCahn", prefix="hmc_bpinn_table",
    loss=baseline_data_loss
)(
    grid_test, hmc_bpinn_uncal_predset, hmc_bpinn_cp_cal_predset, 
    true_solution, 
    hmc_bpinn_df_uncal, hmc_bpinn_df_cal,
    title="2D UQ", main_title="HMC Model Metrics", 
    X_vis=X_train, Y_vis=Y_train
)

<Figure size 640x480 with 0 Axes>